# Maneuver Generation 
##### This notebook guides you through designing an orbit-raising maneuver using a custom strategy focused on thrusting at the orbit nodes. You will learn how to set up orbital models, configure maneuver parameters, and analyze the results. The process includes defining the satellite's initial state from TLE data, configuring the propagation context, and generating the maneuvers to achieve the desired orbital adjustments.

## Jupyter Notebook Help Guide
Welcome to your Jupyter Notebook! If you need assistance or more information about any function, method, or object, the contextual help window is a valuable tool.

***Important - To have access to contextual help, start by running the [Imports](#imports) section below***

![Run Notebook](https://portal.exotrail.space/images/products/notebooks/execute_workflow.jpg)


### How to Open the Contextual Help Window


1. **Use the Help Menu**:
   - Navigate to the top menu bar and click on `Help`. From the dropdown menu, select `Show Contextual Help`.
2. **Keyboard Shortcut**:
   - Press `Shift + Tab` while your cursor is over the code you want more information on. For a more detailed view, press `Shift + Tab` twice.
3. **Using the Inspector**:
   - Type `?` followed by the function or object name and run the cell. For example, `?print`.
   - To see the full documentation, use `??` instead, e.g., `??print`.


### Tips for Using the Contextual Help


- **Inline Help**: Single press `Shift + Tab` to get a brief pop-up of the docstring.
- **Expanded Help**: Double press `Shift + Tab` or click the expand icon in the pop-up to open the full documentation in the help pane.
- **Persistent Help Pane**: Use the Help menu or `Shift + Tab` twice to dock the help pane on the right side of the screen, where it can stay open as you work.
This feature can help you understand function parameters, return types, and see example usages directly in your notebook. Happy coding!

## Imports

In [ ]:
import pandas as pd
import plotly.express as px

from fds.models.maneuvers.strategy import ManeuverStrategy
from fds.models.maneuvers.use_case import ManeuverGeneration
from fds.models.orbital_state import PropagationContext, OrbitalState
from fds.models.spacecraft import Battery, SolarArray, ThrusterElectrical, SpacecraftBox
from fds.models.two_line_element import TwoLineElement

*Note*: If you are using this notebook locally, use the following methods to configure your credentials:

```python
from fds.config import set_client_id, set_client_secret
set_client_id('CLIENT_ID')
set_client_secret('CLIENT_SECRET')
```

## Create Models

### Create Orbital State
`OrbitalState` is the object used in spacetower to group information on the initial orbit, the propagation context and the spacecraft definition.  

#### TLE

The TLE provides the orbital parameters needed for accurate satellite tracking and prediction.

In [ ]:
# Define the initial state from a public TLE
tle = TwoLineElement(
    "1 58295U 23174AR  24137.17541737  .00005775  00000-0  29855-3 0  9994",
    "2 58295  97.4580 211.9750 0010360 352.3995   7.7079 15.16607347 28836"
)

print(f"TLE date (UTC): {tle.date}")

#### Propagation Context

The `PropagationContext` object encapsulates the settings necessary to accurately propagate the satellite's orbit by accounting for gravitational and non-gravitational perturbations. Adjusting these parameters allows for fine-tuning the balance between computational efficiency and the accuracy of the simulation.

In [ ]:
propagation_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=10,
    model_earth_potential_ord=10,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

#### Spacecraft

Here we are using the `SpacecraftBox` class to define our spacecraft.  
To define a simpler model, try using the `SpacecraftSphere` class where battery, solar-arrays and thrusters do not need to be defined.

In [ ]:
battery = Battery(
    depth_of_discharge=0.3,  # 0<x<1
    nominal_capacity=560,  # W
    minimum_charge_for_firing=0.9,  # 0<x<1
    initial_charge=1,  # 0<x<1
)

solar_array = SolarArray(
    kind="DEPLOYABLE_FIXED",
    initialisation_kind=SolarArray.InitialisationKind.MAXIMUM_POWER,
    efficiency=.293,  # 0<x<1
    normal_in_satellite_frame=(0, 0, -1),  # Unit vector
    maximum_power=300,  # W
    surface=0.75,  # m^2
)

electrical_thruster = ThrusterElectrical(
    isp=1100,  # s
    thrust=0.005,  # N
    axis_in_satellite_frame=(-1, 0, 0),  # Unit vector
    propellant_mass=1,  # kg
    wet_mass=13,  # kg
    warm_up_duration=240,  # s
    maximum_thrust_duration=1200,  # s
    impulse=10780,  # Ns
    power=300,  # W
    stand_by_power=1.1,  # W
    warm_up_power=10,  # W
)

spacecraft = SpacecraftBox(
    battery=battery,
    thruster=electrical_thruster,
    solar_array=solar_array,
    platform_mass=90,  # kg
    drag_coefficient=2.2,
    length_x=1,  # m
    length_y=1,  # m
    length_z=1,  # m
    max_angular_velocity=2,  # deg/s
    max_angular_acceleration=.5,  # deg/s^2
)



#### Orbital State

In [ ]:
orbital_state = OrbitalState.from_tle(
    tle=tle,
    covariance_matrix=None,
    propagation_context=propagation_context,
    spacecraft=spacecraft,
)

### Create Maneuver Generation Strategy

In this section, we define the parameters for the orbit-raising maneuver. This includes the position and number of thrust arcs, initialization method, thrust and rest orbits, duty cycle, thrust duration, and whether to stop thrusting during eclipses.

In [ ]:
maneuver_strategy = ManeuverStrategy(
    thrust_arcs_position="ASCENDING_AND_DESCENDING_NODES",
    thrust_arcs_number="TWO",
    thrust_arc_initialisation_kind="DUTY_CYCLE",
    number_of_thrust_orbits=1,
    number_of_rest_orbits=2,
    number_of_shift_orbits=1,
    orbital_duty_cycle=0.2,
    thrust_arc_duration=900,
    stop_thrust_at_eclipse=False
)

## Build & Run Use Case

In [ ]:
mg = ManeuverGeneration(
    initial_orbital_state=orbital_state,
    strategy=maneuver_strategy,
    delta_semi_major_axis=.2,  # km
    delta_eccentricity=0,
    delta_inclination=0.0,
    quaternion_step=60,
    maximum_duration=10 * 24 * 60 * 60  # seconds
)

# Run the maneuver generation
mg.run()

## Results/Post-Processing

### Extract Results

In [ ]:
res_mg = mg.result

c_df, d_df = res_mg.export_roadmap_data_for_dataframe()
c_g_th, d_g_th = res_mg.export_thruster_data_for_gantt()
c_g_at, d_g_at = res_mg.export_attitude_data_for_gantt()

df_all = pd.DataFrame(d_df, columns=c_df)

df_thruster = pd.DataFrame(d_g_th, columns=c_g_th)
df_attitude = pd.DataFrame(d_g_at, columns=c_g_at)

### Plot Results

#### Chronogram corresponding to the "Thruster Modes" and "Attitude Modes"

In [ ]:
fig = px.timeline(df_thruster, x_start="Start", x_end="End", color="Mode", y="Mode")
fig.show()

fig = px.timeline(df_attitude, x_start="Start", x_end="End", color="Mode", y="Mode")
fig.show()

### Print Maneuver Roadmap

In [ ]:
print("Maneuver Roadmap")
df_all